## Estimating bias in topology-change waiting distances under the MS-SMC' model

The solution for the probability that a recombination event *does not change the topology* is exact, however, the expected waiting distance until a topology change event occurs calculated from this probability is not exact, it is approximation. This is because tree-changes (changes to coalescent times but not the topology) can occur during the waiting distance until a topology change. Here we investigate potential biases from this approximation. 


### Approach 1: Analytical solution/approximation
1. Simulate many unlinked genealogies (each for a single site).
2. Calculate `Prob(topology-unchanged | S,G)` under MS-SMC' w/ bias for each one.
3. Calculate the `E[waiting distance to topology-change]` from each prob.

### Approach 2: Estimation from simulations (slow) 
1. Simulate many unlinked loci of long length and get distance to first topology-change event in each.

In [1]:
from concurrent.futures import ProcessPoolExecutor
from typing import Tuple
import numpy as np
import pandas as pd
import ipcoal
import toyplot
import toytree
from scipy import stats

### Parameters

In [66]:
RECOMB = 2e-9
NSPECIES = 8
NSAMPLES = 1    # samples per species
SPECIES_TREE_HEIGHT = 1e6
NEFF_MIN = 50_000
NEFF_MAX = 250_000
NEFF_VALS = 11
SEED = 123
NLOCI = 10       # very small, increased after testing

In [3]:
# get a balanced species tree
sptree = toytree.rtree.baltree(NSPECIES, treeheight=SPECIES_TREE_HEIGHT)
sptree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6d83e0114e564508ba4def00b18bd0f5"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 r0 r1 r2 r3 r4 r5 r6 r7 0 500000 1000000

### Approach 1: get waiting distances to topology change using MS-SMC' analytical solution:

This uses the method we describe in the paper as our analytical solution for the probability of a topology change given a species tree, genealogy, and recombination rate. The expected waiting distance is then calculated from this probability. Here we return arrays for the probability of topology-unchanged, and the expected waiting distance until topology change, for many genealogies. The results will be compared with simulated expectations below.

In [177]:
def get_topo_change_approx(sptree: toytree.ToyTree, neff: int) -> Tuple[np.ndarray, np.ndarray]:
    """Return Prob(topology-unchanged | S,G) for many genealogies.
    
    This uses global variables.
    """
    # init a coalescent model using the species tree and Ne value
    model = ipcoal.Model(sptree, Ne=neff, seed_trees=SEED)
    
    # simulate NLOCI unlinked genealogies
    model.sim_trees(nloci=NLOCI, nsites=1)
    imap = model.get_imap_dict()
    
    # load the first genealogy of every locus as a multitree
    mtree = toytree.mtree(model.df[model.df.tidx == 0].genealogy)
    
    # get Prob(topo-unchanged) 
    probs = np.array([
        ipcoal.smc.get_probability_of_topology_change(model.tree, i, imap)
        for i in mtree
    ])
    
    # get sum branch lengths of each genealogy
    sumlens = np.array([sum(b.dist for b in i if not b.is_root()) for i in mtree])

    # get rate parameters
    rates = sumlens * probs * RECOMB
    
    # get waiting distances
    return probs, np.array([stats.expon.freeze(scale=1 / i).mean() for i in rates])

In [181]:
probs, dists = get_topo_change_approx(sptree, 200_000)

In [182]:
probs

array([0.19447408, 0.31057149, 0.32870847, 0.4011096 , 0.24962485,
       0.26176764, 0.1826206 , 0.35686763, 0.16845868, 0.28238945])

In [183]:
dists

array([275.92672256, 257.18816974, 219.05139216, 193.72002689,
       221.58258607, 249.30500818, 461.89980672, 197.28909908,
       312.57157086, 257.35567948])

### Approach 2: get waiting distances to topology change estimated from simulations:

The expected waiting distance for a topology-change under a given MSC model can also be estimated by using simulations. Here we simulate NLOCI tree sequences that each include
at least one recombination event. The first recombination event at each locus can be 
either a no-change, tree-change not affecting topology, or topology-change event. 
For each locus the probability that one of these three events occurs first should be
equal to their relative probability. Therefore, we simply compute the distance until a topology-change event occurs for each locus.

In [184]:
def get_distance_to_topo_change(series: pd.Series) -> int:
    """Return length until topology-change occurred."""
    start = toytree.tree(series.genealogy.iloc[0])
    sidx = start.get_topology_id(exclude_root=False)
    for idx in series.index:
        other = toytree.tree(series.genealogy[idx])
        odx = other.get_topology_id(exclude_root=False)
        if sidx != odx:
            return series.start[idx]
    raise Exception("no topology changes observed.")

In [185]:
def get_topo_change_from_sims(sptree: toytree.ToyTree, neff: int, nsites: int) -> float:
    """Return the proportion of simulated loci in which a topology-unchanged event 
    occured first.
    """
    # init coalescent model
    model = ipcoal.Model(sptree, Ne=neff, seed_trees=SEED, record_full_arg=False)
    
    # simulate many unlinked loci containing many linked genealogies
    model.sim_trees(nloci=NLOCI, nsites=nsites)
    
    # get lengths until topology-change (raises exception if none present)
    dists = model.df.groupby("locus").apply(get_distance_to_topo_change)
    return dists.values

In [186]:
NLOCI = 10

In [187]:
get_topo_change_from_sims(sptree, 50_000, 500_000)

array([12513, 14608, 78988, 19118,  2793,  3888, 11265, 21402,  4976,
       28107])

In [188]:
get_topo_change_from_sims(sptree, 500_000, 50_000)

array([ 44, 262, 481,  10, 184,  37,  47, 241, 223, 324])

## Compare results of the two methods

In [250]:
# get array to store results of all reps
NLOCI = 10_000
res = np.zeros((NLOCI, NEFF_VALS, 3))
nes = np.linspace(NEFF_MIN, NEFF_MAX, NEFF_VALS).astype(int)
nsites = np.logspace(6, 4, NEFF_VALS).astype(int)

In [251]:
# run jobs in parallel to fill array
rasyncs = {}
with ProcessPoolExecutor(max_workers=6) as pool:
    for nidx in range(res.shape[1]):
        args = (sptree, nes[nidx], nsites[nidx])
        rasyncs[(nidx, 0)] = pool.submit(get_topo_change_approx, *args[:-1])
        rasyncs[(nidx, 1)] = pool.submit(get_topo_change_from_sims, *args)

    for name, future in rasyncs.items():
        nidx = name[0]
        if name[1] == 0:
            parr, darr = future.result()
            res[:, nidx, 0] = parr
            res[:, nidx, 1] = darr
        else:
            res[:, nidx, 2] = future.result()        

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [252]:
# enter results into a dataframe
data = pd.DataFrame(
    data={
        'prob': res[:, :, 0].mean(axis=0),
        'dist_apx': res[:, :, 1].mean(axis=0),
        'dist_sim': res[:, :, 2].mean(axis=0),
        'prob_2.5': np.percentile(res[:, :, 0], 0.025, axis=0),
        'dist_apx_2.5': np.percentile(res[:, :, 1], 0.025, axis=0),
        'dist_sim_2.5': np.percentile(res[:, :, 2], 0.025, axis=0),
        'prob_97.5': np.percentile(res[:, :, 0], 0.975, axis=0),
        'dist_apx_97.5': np.percentile(res[:, :, 1], 0.975, axis=0),
        'dist_sim_97.5': np.percentile(res[:, :, 2], 0.975, axis=0),  
    },
    index=nes,
)

In [253]:
# show the dataframe of results
data

,prob,dist_apx,dist_sim,prob_2.5,dist_apx_2.5,dist_sim_2.5,prob_97.5,dist_apx_97.5,dist_sim_97.5
50000,0.023602,8483.612771,0.0,0.004864,408.875197,0.0,0.00559,565.088576,0.0
70000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0
90000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0
110000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0
130000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0
150000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0
170000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0
190000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0
210000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0
230000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.0


In [ ]:
# save the dataframe
data.to_csv("./validation-topology.csv")

### Plot the Prob(topology-unchanged) expectation versus approximation
We expect bias to be greatest at low Ne, where many tree changes happen for every topology-change.

In [16]:
canvas = toyplot.Canvas(width=400, height=300)
axes = canvas.cartesian(
    xlabel="Effective population size", 
    ylabel="Prob(topology-unchanged | S,G)",
    margin=65,
)
axes.fill(
    probs.index,
    probs.w_bias_mean - probs.w_bias_std,
    probs.w_bias_mean + probs.w_bias_std,
    opacity=0.33,
)
axes.plot(probs.index, probs.w_bias_mean, stroke_width=2, color=toytree.color.COLORS2[0])
marks = [
    axes.scatterplot(probs.index, probs.w_bias_mean, size=8, color=toytree.color.COLORS2[0]),
    axes.scatterplot(probs.index, probs.wo_bias, size=7,
                     marker='s', opacity=0.8, color='black',#toytree.color.COLORS2[1], 
                     mstyle={"stroke": "none"}),
]
for ax in (axes.x, axes.y):
    ax.domain.show = False
    ax.ticks.show = True
    ax.ticks.near = 5
    ax.ticks.far = 0
    ax.ticks.style["stroke-width"] = 1.5
    ax.ticks.labels.offset = 10
    ax.ticks.labels.style["font-size"] = 12
    ax.ticks.locator = toyplot.locator.Extended(6)
    ax.label.offset = 30
    ax.label.style["font-size"] = 14
    ax.spine.style["stroke-width"] = 1.5
canvas.legend(
    bounds=("50%", "80%", "20%", "35%"),
    entries=[
        ("approximation", marks[0]),
        ("exact solution", marks[1]), 
    ],
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="300.0px" viewBox="0 0 400.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9a027bbb2db34b28aff6d27e323bcdff"> <polygon points="75.8,68.73518844040672 101.72,76.0998190524289 127.64,98.8515963159136 153.56,124.17139067946242 179.48,153.0662095030851 205.4,173.00658518899178 231.32,190.23860721972832 257.24,204.50283207164813 283.16,213.1813213478685 309.08000000000004,226.21119224064103 335.0,235.0 335.0,175.88802221479332 309.08000000000004,167.03528209634604 283.16,155.36418766603657 257.24,147.2794933782837 231.32,133.58912245339548 205.4,116.63042459233188 179.48,101.16550944520193 153.56,80.70185576680018 127.64,67.82718257983835 101.72,65.0 75.8,68.73518275438428" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 0 50000 100000 150000 200000 250000 Effective population size 0.6 0.7 0.8 0.9 1.0 Prob(topology-unchanged | S,G) approximation exact solution

In [17]:
import toyplot.svg
# toyplot.svg.render(canvas, "../manuscript/figures/approximation.svg")